In [1]:
import pygame
import numpy as np

pygame 2.1.2 (SDL 2.0.20, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class World:
   def __init__(self, food, forage_pheromone, home_pheromone, nest = 0 ):
      self.food = food
      self.forage = forage_pheromone
      self.home = home_pheromone
      self.nest = nest
      self.bound = home_pheromone # originally i tried expanding the pheromones so this is an artifact.
      self.size = 80

      self.Home_evapoaration = 0.3
      self.Foraging_evaporation = 0.05
      
   def evaporate(self):
      self.forage *= self.Foraging_evaporation
      self.home *= self.Home_evapoaration

In [3]:
class Ant:
    def __init__(self, direction, X, Y, food = 0):
        self.X = X
        self.Y = Y
        self.dir = direction
        self.food = food

    def _calculate_next_positions(self, offset):
        return [(self.Y + direrctions[(self.dir + i + 8) % 8][1], self.X + direrctions[(self.dir + i + 8) % 8][0]) for i in offset]
    def _calculate_next_position(self, direction):
        return self.Y + direrctions[direction % 8][1], self.X + direrctions[direction % 8][0]

    def move(self, world):
       # Check pheromones depending on whether the ant has food
        pher = world.home if self.food > 0 else world.forage
        boundries = world.bound

        next_positions = self._calculate_next_positions([-1, 0, 1])

        # Initialize prob_vals and tile arrays with default values
        prob_vals = [0, 0, 0]
        tile = [None, None, None]
        for i in range(3):
            Y, X = next_positions[i]
            if 0 <= Y < world.size and 0 <= X < world.size:
                prob_vals[i] = prob(pher[Y, X])
                tile[i] = boundries[Y, X]

        random_choice = np.random.rand() 
        if random_choice < 0.05:  # this little thing keeps ants from getting stuck on high concentration pheromone cells
            self.dir = (self.dir + np.random.choice([-1, 0, 1])) % 8
        elif any(tile):  # if hit any boundary. this function is present cause my ants keep geting stuck on edges
            available_directions = [i for i, t in enumerate(tile) if not t]
            if not available_directions:  
                return  
            new_dir = np.random.choice(available_directions)
            self.dir = (self.dir + new_dir - 1) % 8   
        else:
            if sum(prob_vals) == 0:
                self.dir = (self.dir + np.random.choice([-1, 0, 1])) % 8
            else:
                self.dir = (self.dir + np.random.choice([-1, 0, 1], p=[p/sum(prob_vals) for p in prob_vals])) % 8

        next_Y, next_X = self._calculate_next_position(self.dir)

        # we move 
        if 0 <= next_Y < world.size and 0 <= next_X < world.size:  # last check if in bounds. 
            self.Y = next_Y
            self.X = next_X

    def mark_pheromone(self, world):
        if 0 <= self.Y < world.size and 0 <= self.X < world.size:
            if self.food > 0 and (self.Y > 10 or self.X > 10):
                world.home[self.Y, self.X] += 1
            if self.food == 0 and (self.Y > 10 or self.X > 10):
                world.forage[self.Y, self.X] += 1

    def take_food(self, world):
        next_positions = self._calculate_next_positions([-1, 0, 1])
        food = world.food
        tiles = [None, None, None]

        # we look at food in frontal cells
        for i in range(3):
            Y, X = next_positions[i]
            if 0 <= Y < world.size and 0 <= X < world.size:
                tiles[i] = food[Y, X]

        for i, tile in enumerate(tiles):
            if tile and tile > 0:
                Y, X = next_positions[i]
                world.food[Y, X] -= 1
                self.food += 1
                self.dir = (self.dir + i - 1 + 4) % 8  # turn around
                self.Y, self.X = self._calculate_next_position(self.dir) # move. here we do a little sequence
                return

    def deposit_food(self, world):
        if (0 < self.X < 10) and (0 < self.Y < 10) and self.food > 0:
            self.food = 0
            world.nest += self.food
            self.food = 0

In [4]:
direrctions = [[1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1], [0, 1], [1, 1]]

def prob(I): # i decided to use formula given on the presentation
   return (2+I)**(7)

In [5]:
# we quickly define colors to make it easier
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREY = (128, 128, 128)

# Set window parameters for pygame
WINDOW_SIZE = [800, 800]
CELL_SIZE = 10  # size of one cell. we will need it later for drawing pheromones and ants

In [6]:
# we spawn our ants and make start at the edge of the nest
Ants = [Ant(np.random.randint(0,8), 10,10) for _ in range(40)] 


food = np.zeros((80, 80)) # to create food
forage_pheromone = np.zeros((80, 80))# to create pheromoen grids
home_pheromone = np.zeros((80, 80))

# Place food
food[70:80, 70:80] = 1 # how much food points in one cell, ant can carry only one point  

# food, forage_pheromone, home_pheromone, boundries
world = World(food, forage_pheromone, home_pheromone)

In [7]:
# Initialize Pygame
pygame.init()

screen = pygame.display.set_mode(WINDOW_SIZE)
clock = pygame.time.Clock() #makes it easier to speed up the simulation

while True: 
    for event in pygame.event.get(): #basic setup
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            pygame.quit()
            quit()

    for ant in Ants:
        ant.move(world)# we make them move
        ant.take_food(world) # let them take food if they can
        ant.mark_pheromone(world) # let them mark pheromones. they depend on having food
        ant.deposit_food(world) #if they can, let them deposit food
    
    world.evaporate() # we evaporate the pheromones

    screen.fill((0,0,0)) # needs of pygame, we set background to black

    # we draw values of pheromones in cells. green is for having food and blue for searching for food
    for i in range(80):
        for j in range(80):
            if world.home[i][j] > 0:
                pygame.draw.rect(screen, (0,126,0), [j*CELL_SIZE, i*CELL_SIZE, CELL_SIZE, CELL_SIZE])
            if world.forage[i][j] > 0:
                pygame.draw.rect(screen, (0,0,126), [j*CELL_SIZE + CELL_SIZE//4 , i*CELL_SIZE+ CELL_SIZE//4 , CELL_SIZE//2, CELL_SIZE//2])

    # draw ants
    for ant in Ants:
        pygame.draw.rect(screen, RED, [ant.X*CELL_SIZE, ant.Y*CELL_SIZE, CELL_SIZE, CELL_SIZE])

    # draw food
    for i in range(69,80):
        for j in range(69,80):
            if world.food[i][j] > 0:
                pygame.draw.rect(screen, GREEN, [j*CELL_SIZE, i*CELL_SIZE, CELL_SIZE, CELL_SIZE])
                
    # draw nest
    for i in range(11):
        for j in range(11):
            pygame.draw.rect(screen, BLUE, [j*CELL_SIZE, i*CELL_SIZE, CELL_SIZE, CELL_SIZE])
    
    pygame.display.flip()# simple setup
    clock.tick(200)#we speed up


error: display Surface quit

: 